# Limits

In [ ]:
using CalculusWithJulia
using CalculusWithJulia.WeaveSupport
using Plots
nothing

An historic problem in the history of math was to find the area under
the graph of $f(x)=x^2$ between $[0,1]$.



There isn't a ready-made formula for the area of this
shape, as there is for a triangle or a square. However,
[Archimedes](http://en.wikipedia.org/wiki/The_Quadrature_of_the_Parabola)
found a method to compute areas enclosed by a parabola and line
segments that cross the parabola.

In [ ]:
###{{{archimedes_parabola}}}
pyplot()
fig_size=(400, 300)

f(x) = x^2
colors = [:black, :blue, :orange, :red, :green, :orange, :purple]

## Area of parabola
function make_triangle_graph(n)
    title = "Area of parabolic cup ..."
    n==1 && (title = "\${Area = }1/2\$")
    n==2 && (title = "\${Area = previous }+ 1/8\$")
    n==3 && (title = "\${Area = previous }+ 2*(1/8)^2\$")
    n==4 && (title = "\${Area = previous }+ 4*(1/8)^3\$")
    n==5 && (title = "\${Area = previous }+ 8*(1/8)^4\$")
    n==6 && (title = "\${Area = previous }+ 16*(1/8)^5\$")
    n==7 && (title = "\${Area = previous }+ 32*(1/8)^6\$")



    plt = plot(f, 0, 1, legend=false, size = fig_size, linewidth=2)
    annotate!(plt, [(0.05, 0.9, text(title,:left))])  # if in title, it grows funny with gr
    n >= 1 && plot!(plt, [1,0,0,1, 0], [1,1,0,1,1], color=colors[1], linetype=:polygon, fill=colors[1], alpha=.2)
    n == 1 && plot!(plt, [1,0,0,1, 0], [1,1,0,1,1], color=colors[1], linewidth=2)
    for k in 2:n
        xs = range(0,stop=1, length=1+2^(k-1))
        ys = map(f, xs)
        k < n && plot!(plt, xs, ys, linetype=:polygon, fill=:black, alpha=.2)
        if k == n
            plot!(plt, xs, ys, color=colors[k], linetype=:polygon, fill=:black, alpha=.2)
            plot!(plt, xs, ys, color=:black, linewidth=2)
        end
    end
    plt
end


n = 7
anim = @animate for i=1:n
    make_triangle_graph(i)
end

imgfile = tempname() * ".gif"
gif(anim, imgfile, fps = 1)


caption = L"""
The first triangle has area $1/2$, the second has area $1/8$, then $2$ have area $(1/8)^2$, $4$ have area $(1/8)^3$, ...
With some algebra, the total area then should be $1/2 \cdot (1 + (1/4) + (1/4)^2 + \cdots) = 2/3$.
"""

plotly()
ImageFile(imgfile, caption)

The figure illustrates a means to compute the area bounded by the
parabola, the line $y=1$ and the line $x=0$ using triangles. It
suggests that this area can be found by adding the following sum

$$~
A = 1/2 + 1/8 + 2 \cdot (1/8)^2 + 4 \cdot (1/8)^3 + \cdots
~$$

This value is $2/3$, so the area under the curve would be
$1/3$. Forget about this specific value - which through more modern machinery becomes uneventful - and focus for a minute on the
method: a problem is solved by a suggestion of an infinite process, in
this case the creation of more triangles to approximate the
unaccounted for area. This is the so-call method of
[exhaustion](http://en.wikipedia.org/wiki/Method_of_exhaustion) known
since the 5th century BC. Archimedes used this method to solve a wide
range of area problems related to basic geometric shapes, including a
more general statement of what we described above.

The $\cdots$ in the sum expression are the indication that this process continues and that
the answer is at the end of an *infinite* process. To make this line of
reasoning rigorous requires the concept of a limit. The concept of a
limit is then an old one, but it wasn't until the age of calculus
that it was formalized.

[Fermat](http://en.wikipedia.org/wiki/Adequality) in the 1600s
essentially took a limit to find the slope of a tangent line to a
polynomial curve. Newton in the late 1600s, exploited the idea in his
development of calculus (as did Leibniz). Yet it wasn't until the
1800s that
[Bolzano](http://en.wikipedia.org/wiki/Limit_of_a_function#History),
Cauchy and Weierstrass put the idea on a firm footing.


To make things more precise, we begin by discussing the limit of a univariate function as $x$ approaches $c$.

Informally, if a limit exists it is the value that $f(x)$ gets close to as $x$ gets close to - but not equal to - $c$.

The modern formulation is due to Weirstrass:

> The limit of $f(x)$ as $x$ approaches $c$ is $L$ if for every real $\epsilon > 0$,
> there exists a real $\delta > 0$ such that for all real $x$, $0 < \lvert x − c \rvert < \delta$
> implies $\lvert f(x) − L \rvert < \epsilon$. The notation used is $\lim_{x \rightarrow c}f(x) = L$.

We comment on this later.


Cauchy begins his incredibly influential
[treatise](http://gallica.bnf.fr/ark:/12148/bpt6k90196z/f25.image) on
calculus considering two examples, the limit as $x$ goes to $0$ of

$$~ \frac{\sin(x)}{x} \quad\text{and}\quad (1 + x)^{1/x}.  ~$$

These take the indeterminate forms $0/0$ and $1^\infty$, which are
found by just putting $0$ in for $x$. An expression does not need to
be defined at $c$, as these two aren't, to discuss its limit. Cauchy
illustrates two methods to approach the questions above. The first is
to pull out an inequality:

$$~
\frac{\sin(x)}{\sin(x)} > \frac{\sin(x)}{x} > \frac{\sin(x)}{\tan(x)}.
~$$

This bounds the expression $\sin(x)/x$ between $1$ and $\cos(x)$ and as $x$ gets close to $0$, the value of $\cos(x)$ "clearly" goes to $1$, hence $L$ must be $1$. This is an application of the squeeze theorem.

To discuss the case of $(1+x)^{1/x}$, Cauchy first resorts to log tables
to illustrate an approximate value. We can use `Julia` to find his
approximate value:

In [ ]:
x = 1/10000
(1 + x)^(1/x)

A table can show the progression to this value:

In [ ]:
f(x) = (1 + x)^(1/x)
xs = [1/10^i for i in 1:5]
[xs f.(xs)]

However viewed, this is an approximate value for the actual answer - which is $e$. Cauchy
demonstrates this value with a fair amount of work.

These two cases illustrate that though the definition of the limit
exists, the computation of a limit is generally found by other means
and the intuition of the value of the limit can be gained numerically.

### Indeterminate forms

First it should be noted that for most of the functions encountered, the concepts of a limit at a typical point $c$ is nothing more than just function evaluation at $c$. This is because, at a typical point, the functions are nicely behaved. However, most questions asked about limits find points that are not typical. For these, the result of evaluating the function at $c$ is typically undefined, and the value comes in one of several *indeterminate forms*: $0/0$, $\infty/\infty$, $0 \cdot \infty$, $\infty - \infty$, $0^0$, $1^\infty$, and $\infty^0$.

`Julia` can help - at times - identify these indeterminate forms, as many such operations produce `NaN`. For example:

In [ ]:
0/0, Inf/Inf, 0 * Inf, Inf - Inf

However, the values with powers generally do not help, as the IEEE standard has `0^0` evaluating to 1:

In [ ]:
0^0, 1^Inf, Inf^0

However, this can be unreliable, as floating point issues may mask the true evaluation. However, as a cheap trick it can work. So, the limit as $x$ goes to $1$ of $\sin(x)/x$ is simply found by evaluation:

In [ ]:
x = 1
sin(x) / x

But at $x=0$ we get an indicator that there is an issue with just evaluating the function:

In [ ]:
x = 0
sin(x) / x

The above is really just a heuristic. For some functions this is just
not true. For example, the $f(x) = \sqrt{x}$ is only defined on $[0,
\infty)$ There is technically no limit at $0$, per se, as the function
is not defined around $0$. Other functions jump at values, and will
not have a limit, despite having well defined values. The `floor`
function is the function that rounds down to the nearest integer. At
integer values there will be a jump, even though the function is
defined.

## Graphical approaches to limits

Let's return to the function $f(x) = \sin(x)/x$. This function was studied by Euler as part of his solution to the [Basel](http://en.wikipedia.org/wiki/Basel_problem) problem. He knew that near $0$, $\sin(x) \approx x$, so the ratio is close to $1$ if $x$ is near $0$. Hence, the intuition is $\lim_{x \rightarrow 0} \sin(x)/x = 1$, as Cauchy wrote. We can verify this limit graphically two ways. First, a simple graph shows no issue at $0$:

In [ ]:
using CalculusWithJulia  # loads SymPy
using Plots
f(x) = sin(x)/x
plot(f, -pi/2, pi/2)

The $y$ values of the graph seem to go to $1$ as the $x$ values get
close to 0. (That the graph looks defined at $0$ is due to the fact
that the points sampled to graph do not include $0$. Otherwise, since
`f(0)` is `NaN`, there would be no lines connecting to the value with
`x=0`.)

We can also verify Euler's intuition through this graph:

In [ ]:
plot([sin, x -> x], -pi/2,  pi/2)

That the two are indistinguishable near $0$ makes it easy to see that their ratio should be going towards $1$.


The graphical approach to limits - plotting $f(x)$ around $c$ and
observing if the $y$ values seem to converge to an $L$ value when $x$
get close to $c$ - allows us to gather quickly if a function seems to
have a limit at $c$, though the precise value of $L$ may be hard to identify.

##### Example


Consider now the following limit

$$~
\lim_{x \rightarrow 2} \frac{x^2 - 5x + 6}{x^2 +x - 6}
~$$

Noting that this is a ratio of nice polynomial functions, we first
check whether there is anything to do:

In [ ]:
f(x) = (x^2 - 5x + 6) / (x^2 + x - 6)
c = 2
f(c)

The `NaN` indicates that this function is indeterminate at $c=2$. A
quick plot gives us an idea that the limit exists and is roughly
$-0.2$:

In [ ]:
c, delta = 2, 1
plot(f, c - delta, c + delta)

The graph looks "continuous." In fact, the value $c=2$ is termed a
*removable singularity* as redefining $f(x)$ to be $-0.2$ when
$x=2$ results in a "continuous" function.

As an aside, we can redefine `f` using the "ternary operator":

In [ ]:
f(x) = x == 2.0 ? -0.2 :  (x^2 - 5x + 6) / (x^2 + x - 6)

This particular case is a textbook example: one can easily factor
$f(x)$ to get:

$$~
f(x) = \frac{(x-2)(x-3)}{(x-2)(x+3)}
~$$

Written in this form, we clearly see that this is the same function as
$g(x) = (x-3)/(x+3)$ when $x \neq 2$. The function $g(x)$ is
"continuous" at $x=2$. So were one to redefine $f(x)$ at $x=2$ to be
$g(2) = (2 - 3)/(2 + 3) = -0.2$ it would be made continuous, hence the
term removable singularity.



## Numerical approaches to limits

The investigation of $\lim_{x \rightarrow 0}(1 + x)^{1/x}$ by
evaluating the function at $1/10000$ by Cauchy can be done much more easily
nowadays. As does a graphical approach, a numerical approach can
give insight into a limit and often a good numeric estimate.

The basic idea is to create a sequence of $x$ values going towards $c$
and then investigate if the corresponding $y$ values are eventually near some
$L$.

Best, to see by example. Suppose we are asked to investigate

$$~
\lim_{x \rightarrow 25} \frac{\sqrt{x} - 5}{\sqrt{x - 16} - 3}.
~$$

We first define a function and check if there are issues at 25:

In [ ]:
f(x) = (sqrt(x) - 5) / (sqrt(x-16) - 3)
c = 25
f(c)

So yes, an issue of the indeterminate form $0/0$. We investigate numerically by making a set of numbers getting close to $c$. This is most easily done making numbers getting close to $0$ and adding them to or subtracting them from $c$. Some natural candidates are negative powers of 10:

In [ ]:
hs = [1/10^i for i in 1:8]

We can add these to $c$ and then evaluate:

In [ ]:
xs = c .+ hs
ys = f.(xs)

To visualize, we can put in a table using `[xs ys]` notation:

In [ ]:
[xs ys]

The $y$-values seem to be getting near $0.6$.

Since limits are defined by the expression $0 < \lvert x-c\rvert < \delta$, we should also look at values smaller than $c$. There isn't much difference (note the `.-` sign in `c .- hs`):

In [ ]:
xs = c .- hs
ys = f.(xs)
[xs ys]

Same story. The numeric evidence supports a limit of $L=0.6$.

##### Example, the secant line

Let $f(x) = x^x$ and consider the ratio:

$$~
\frac{f(c + h) - f(c)}{h}
~$$

As $h$ goes to $0$, this will take the form $0/0$ in most cases, and
in the particular case of $f(x) = x^x$ and $c=1$ it will be. The
expression has a geometric interpretation of being the slope of the
secant line connecting the two points $(c,f(c))$ and $(c+h, f(c+h))$.

To look at the limit in this example, we have (recycling the values in `hs`):

In [ ]:
c = 1
f(x) = x^x
ys = [(f(c + h) - f(c)) / h for  h in hs]
[hs ys]

The limit looks like $L=1$. A similar check on the left will confirm this numerically:

In [ ]:
ys = [(f(c + h) - f(c)) / h for  h in -hs]
[-hs ys]

### Issues with the numeric approach

The numeric approach often gives a good intuition as to the existence of a limit and its value. However, it can be misleading. Consider this limit question:

$$~
\lim_{x \rightarrow 0} \frac{1 - \cos(x)}{x^2}.
~$$

We can see that it is indeterminate of the form $0/0$:

In [ ]:
f(x) = (1 - cos(x)) / x^2
f(0)

What is the value of $L$, if it exists? A quick attempt numerically yields:

In [ ]:
c = 0
xs = c .+ hs
ys = [f(x) for x in xs]
[xs ys]

Hmm, the values in `ys` appear to be going to $0.5$, but then end up at
$0$. Is the limit $0$ or $1/2$? The answer is $1/2$. The last $0$ is
an artifact of floating point arithmetic. To investigate, we look more carefully at the two ratios:

In [ ]:
y1s = [1 - cos(x) for x in xs]
y2s = [x^2 for x in xs]
[xs y1s y2s]

Looking at the bottom of the second column reveals the error. The value of `1 - cos(1.0e-8)` is
`0` and not a value around `5e-17`, as would be expected from the pattern above it. This is because the smallest
floating point value less than `1.0` is more than `5e-17` units away,
so `cos(1e-8)` is evaluated to be `1.0`. There just isn't enough
granularity to get this close to $0$.

Not that we needed to. The answer would have been clear if we had
stopped with `x=1e-6`, say.

In general, some functions will frustrate the numeric approach. It is
best to be wary of results. At a minimum they should confirm what a quick
graph shows.

## Symbolic approach to limits

The `SymPy` package provides a `limit` function for finding the limit
of an expression in a given variable. It is loaded above, when the `CalculusWithJulia` package was loaded. The `limit` function's use requires the expression,
the variable and a value for $c$. (Similar to the three things in the
notation $\lim_{x \rightarrow c}f(x)$.)

For example, the limit at $0$ of $(1-\cos(x))/x^2$ is easily handled:

In [ ]:
@vars x real=true
f(x) = (1 - cos(x)) / x^2
limit(f(x), x=>0)		# f(x) is a symbolic expression when x is

The pair notation (`x=>0`) is used to indicate the variable and the value it is going to.

As a convenience, there is also an "operator" version of `limit` for univariate functions, where only a function object and `c` need be specified:

In [ ]:
limit(f, 0)

##### Examples

Find the [limits](http://en.wikipedia.org/wiki/L%27H%C3%B4pital%27s_rule):

$$~
\lim_{x \rightarrow 0} \frac{2\sin(x) - \sin(2x)}{x - \sin(x)}, \quad
\lim_{x \rightarrow 0} \frac{e^x - 1 - x}{x^2}, \quad
\lim_{\rho \rightarrow 0} \frac{x^{1-\rho} - 1}{1 - \rho}.
~$$

We have for the first:

In [ ]:
limit( (2sin(x) - sin(2x)) / (x - sin(x)), x=>0)

The second is similarly done, though here we define a function for variety:

In [ ]:
f(x) = (exp(x) - 1 - x) / x^2
limit(f, 0)

Finally, for the third we define a new variable and proceed:

In [ ]:
@vars rho real=true
limit( (x^(1-rho) - 1) / (1 - rho), rho=>1)

##### Example, floating point conversion issues

The [algorithm](http://www.cybertester.com/data/gruntz.pdf)
implemented in `SymPy` for symbolic limits is quite powerful. However,
some care must be exercised to avoid undesirable conversions from
exact values to floating point values.


To Illustrate, let's look at the limit as $x$ goes to $\pi/2$ of $f(x)
= \cos(x) / (x - \pi/2)$. We follow our past practice:

In [ ]:
c = pi/2
f(x) = cos(x) / (x - pi/2)
f(c)

The value is not `NaN`, rather `Inf`. This is because `cos(pi/2)` is
not exactly $0$ as it should be, as `pi/2` is rounded. This minor
difference is important. If we try and correct for this by using `PI` we have:

In [ ]:
limit(f(x), x=>PI/2)

The value is not right, as this simple graph suggests the limit is in fact $-1$:

In [ ]:
plot(f, c - pi/4, c + pi/4)

The difference between `pi` and `PI` can be significant, and though
usually `pi` is silently converted to `PI`, it doesn't happen here as
the division by `2` happens first, which turns the symbol into an approximate
floating point number. Hence, `SymPy` is giving the correct answer for
the problem it is given, it just isn't the problem we wanted to look
at.

Trying again, being more aware of how `pi` and `PI` differ, we have:

In [ ]:
f(x) = cos(x) / (x - PI/2)
limit(f(x), x => PI/2)

## Rules for limits

The `limit` function doesn't compute limits from the definition,
rather it applies some known facts about functions within a set of
rules. Some of these rules are the following. Suppose the individual limits of $f$ and $g$ always exist below.

In [ ]:
using DataFrames
rules = [L"\lim_{x \rightarrow c} (a \cdot f(x) + b \cdot g(x)) = a \cdot
  \lim_{x \rightarrow c} f(x) + b \cdot \lim_{x \rightarrow c}
  g(x)",
  L"\lim_{x \rightarrow c} f(x) \cdot g(x) = \lim_{x \rightarrow c}
  f(x) \cdot \lim_{x \rightarrow c} g(x)",
  L" $\lim_{x \rightarrow c} f(x) / g(x) = \lim_{x \rightarrow c} f(x) /
  \lim_{x \rightarrow c} g(x)$ - provided $\lim_{x \rightarrow c} g(x) \neq 0$",
  L" $\lim_{x \rightarrow c} (f \circ g)(x) = \lim_{x \rightarrow L}
  f(x)$, where $\lim_{x \rightarrow c}g(x) = L$"]
descr = [""" This says that limits involving sums, differences or scalar
  multiples of functions can be computed by first doing the individual
  limits and then combining the answers.""",
  """This says limits of
  products can be found by computing the limit of the individual
  factors and then combining.""",
  L"""This says limits of ratios can be found by computing
  the limit of the individual terms and then dividing provided you
  don't divide by $0$. The last part is really important, as this rule
  is no help with the common indeterminate form $0/0$.""",
  L"""This says the limit of
  compositions can be found by taking the limit of the interior
  function ($L$) and then finding the limit of the exterior function
  as $x$ approaches $L$."""]
  d = DataFrame(Rule=rules, Description=descr)
  table(d)

These, together with the fact that our basic algebraic functions have limits
that can be found by simple evaluation, mean that many limits are easy to
compute.

##### Example, composition

For example, consider for some non-zero $k$ the following limit:

$$~
\lim_{x \rightarrow 0} \frac{\sin(kx)}{x}.
~$$

This is clearly related to the function $f(x) = \sin(x)/x$, but is in fact the limit of the function $g(x) = k f(kx)$. This then follows:


$$~
\lim_{x \rightarrow 0} \frac{\sin(kx)}{x} = \lim_{x \rightarrow 0} k f(kx) = k \lim_{x \rightarrow 0} f(kx) = k\lim_{x \rightarrow 0} f(x) = k.
~$$

Why does the limit of $f(kx)$ at 0 equal the limit of $f(x)$ at 0?
Well, $f(kx)$ is a composition, $f(h(x))$ where $h(x) = kx$. The limit
of $h$ at 0 is 0, so this is a special case of the rule for
compositions.

Basically when taking a limit as $x$ goes to $0$ we can multiply $x$ by any constant and figure out the limit for that. (It is as though we "go to" $0$ faster or slower. but are still going to $0$. This can be generalized to any function $g(x)$ with a limit of $0$ at $0$: $\lim_{x \rightarrow 0}f(g(x)) = \lim_{x \rightarrow 0}f(x)$.)

##### Example, products

Consider this complicated limit found on this [Wikipedia](http://en.wikipedia.org/wiki/L%27H%C3%B4pital%27s_rule) page.

$$~
\lim_{x \rightarrow 1/2} \frac{\sin(\pi x)}{\pi x} \cdot \frac{\cos(\pi x)}{1 - (2x)^2}.
~$$

We know the first factor has a limit found by evaluation: $2/\pi$, so it is really just a constant. The second we can compute:

In [ ]:
g(x) = cos(PI*x) / (1 - (2x)^2)
limit(g, 1//2)

Putting together, we would get $1/2$. Which we could have done directly in this case:

In [ ]:
limit(sin(PI*x)/(PI*x) * g(x), x=>1//2)

##### Example, ratios

Consider again the limit of $\cos(\pi x) / (1 - (2x)^2)$ at $c=1/2$. A graph of both the top and bottom functions shows the indeterminate, $0/0$, form:

In [ ]:
plot(cos(pi*x), 0.4, 0.6)
plot!(1 - (2x)^2)

However, following Euler's insight that $\sin(x)/x$ will have a limit
at $0$ of $1$ as $\sin(x) \approx x$, and $x/x$ has a limit of $1$ at
$c=0$, we can see that $\cos(\pi x)$ looks like $-\pi\cdot (x - 1/2)$
and $(1 - (2x)^2)$ looks like $-4(x-1/2)$ around $x=1/2$:

In [ ]:
plot(cos(pi*x), 0.4, 0.6)
plot!(-pi*(x - 1/2))

In [ ]:
plot(1 - (2x)^2, 0.4, 0.6)
plot!(-4(x - 1/2))

So around $c=1/2$ the ratio should look like $-\pi (x-1/2) / ( -4(x -	 1/2)) = \pi/4$, which indeed it does, as that is the limit.

This is the basis of L'Hôpital's rule, which we will return to once the derivative is discussed.



## Limits from the definition

The formal definition of a limit involves clarifying what it means for
$f(x)$ to be "close to $L$" when $x$ is "close to $c$". These are
quantified by the inequalities $0 < \lvert x-c\rvert < \delta$ and the $\lvert f(x) -
L\rvert < \epsilon$. The second does not have the restriction that it is
greater than $0$, as indeed $f(x)$ can equal $L$. The order is
important: it says for any idea of close for $f(x)$ to $L$, an idea of close must be found for $x$ to $c$.

The key is identifying a value for $\delta$  for a given value of $\epsilon$.

A simple case is the linear case. Consider the function $f(x) = 3x + 2$. Verify that the limit at $c=1$ is $5$.

We show "numerically" that $\delta = \epsilon/3$.

In [ ]:
f(x) = 3x + 2
c, L = 1, 5
epsilon = rand()                 # some number in (0,1)
delta = epsilon / 3
xs = c .+ delta * rand(100)       # 100 numbers, c < x < c + delta
as = [abs(f(x) - L) < epsilon for x in xs]
all(as)                          # are all the as true?

These lines produce a random $\epsilon$, the resulting $\delta$, and then verify for 100 numbers
within $(c, c+\delta)$ that the inequality $\lvert f(x) - L \rvert < \epsilon$
holds for each. Running them again and again should always produce
`true` if $L$ is the limit and $\delta$ is chosen properly.

(Of course, we should also verify values to the left of $c$.)


If there is worry about the possibility that `epsilon == 0`, we could
replace its definition with `epsilon = max(eps(), rand())`.


In this case, $\delta$ is easy to guess, as the function is linear and
has slope $3$. This basically says the $y$ scale is 3 times the $x$
scale. For non-linear functions, finding $\delta$ for a given
$\epsilon$ can be a challenge. For the function $f(x) = x^3$,
illustrated below, a value of $\delta=\epsilon^{1/3}$ is used for $c=0$:

In [ ]:
## {{{ limit_e_d }}}
pyplot()
function make_limit_e_d(n)
    f(x) = x^3

    xs = range(-.9, stop=.9, length=50)
    ys = map(f, xs)


    plt = plot(f, -.9, .9, legend=false, size=fig_size)
    if n == 0
        nothing
    else
        k = div(n+1,2)
        epsilon = 1/2^k
        delta = cbrt(epsilon)
        if isodd(n)
            plot!(plt, xs, 0*xs .+ epsilon, color=:orange)
            plot!(plt, xs, 0*xs .- epsilon, color=:orange)
        else
            plot!(delta * [-1,  1], epsilon * [ 1, 1], color=:orange)
            plot!(delta * [ 1, -1], epsilon * [-1,-1], color=:orange)
            plot!(delta * [-1, -1], epsilon * [-1, 1], color=:red)
            plot!(delta * [ 1,  1], epsilon * [-1, 1], color=:red)
        end
    end
    plt
end


n = 11
anim = @animate for i=1:n
    make_limit_e_d(i-1)
end

imgfile = tempname() * ".gif"
gif(anim, imgfile, fps = 1)


caption = L"""

Demonstration of $\epsilon$-$\delta$ proof of $\lim_{x \rightarrow 0}
x^3 = 0$. For any $\epsilon>0$ (the orange lines) there exists a
$\delta>0$ (the red lines of the box) for which the function $f(x)$
does not leave the top or bottom of the box (except possibly at the
edges). In this example $\delta^3=\epsilon$.

"""

plotly()
ImageFile(imgfile, caption)

## Questions


###### Question

From the graph, find the limit:

$$~
L = \lim_{x\rightarrow 1}  \frac{x^2−3x+2}{x^2−6x+5}
~$$

In [ ]:
f(x) = (x^2 - 3x +2) / (x^2 - 6x + 5)
plot(f, 0,2)

In [ ]:
ans = 1/4
numericq(ans, 1e-1)

###### Question

From the graph, find the limit $L$:

$$~
L = \lim_{x \rightarrow -2} \frac{x}{x+1} \frac{x^2}{x^2 + 4}
~$$

In [ ]:
f(x) = x/(x+1)*x^2/(x^2+4)
plot(f, -3, -1.25)

In [ ]:
f(x) = x/(x+1)*x^2/(x^2+4)
val = f(-2)
numericq(val, 1e-1)

###### Question

Graphically investigate the limit

$$~
L = \lim_{x \rightarrow 0} \frac{e^x - 1}{x}.
~$$

What is the value of $L$?

In [ ]:
f(x) = (exp(x) - 1)/x
p = plot(f, -1, 1)

In [ ]:
val = N(limit(f, 0))
numericq(val, 1e-1)

###### Question

Graphically investigate the limit

$$~
\lim_{x \rightarrow 0} \frac{\cos(x) - 1}{x}.
~$$

The limit exists, what is the value?

In [ ]:
val = 0
numericq(val, 1e-2)

###### Question

The following limit is commonly used:

$$~
\lim_{h \rightarrow 0} \frac{e^{x + h} - e^x}{h} = L.
~$$

Factoring out $e^x$ from the top and using rules of limits this becomes,

$$~
L = e^x \lim_{h \rightarrow 0} \frac{e^h - 1}{h}.
~$$

What is $L$?

In [ ]:
choices = [L"0", L"1", L"e^x"]
ans = 3
radioq(choices, ans)

###### Question

The following limit is commonly used:

$$~
\lim_{h \rightarrow 0} \frac{\sin(x + h) - \sin(x)}{h} = L.
~$$

The answer should depend on $x$, though it is possible it is a
constant.  Using a double angle formula and the rules of limits, this
can be written as:

$$~
L = \cos(x) \lim_{h \rightarrow 0}\frac{\sin(h)}{h} + \sin(x) \lim_{h \rightarrow 0}\frac{\cos(h)-1}{h}.
~$$

Using the last result, what is the value of $L$?

In [ ]:
choices = [L"\cos(x)", L"\sin(x)", "1", "0", L"\sin(h)/h"]
ans = 1
radioq(choices, ans)

###### Question <small>Squeeze theorem</small>

Let's look at the function $f(x) = x \sin(1/x)$. A graph around $0$
can be made with:

In [ ]:
f(x) = x == 0 ? NaN : x * sin(1/x)
c, delta = 0, 1/4
plot([f, abs, x -> -abs(x)], c - delta, c + delta)

This graph clearly oscillates near $0$. To the graph of $f$, we added
graphs of both $g(x) = \lvert x\rvert$ and $h(x) = - \lvert x\rvert$. From this graph it is
easy to see by the "squeeze theorem" that the limit at $x=0$ is
$0$. Why?

In [ ]:
choices=[L"""The functions $g$ and $h$ both have a limit of $0$ at $x=0$ and the function $f$ is in
between both $g$ and $h$, so must to have a limit of $0$.
""",
	L"The functions $g$ and $h$ squeeze each other as $g(x) > h(x)$",
         L"The function $f$ has no limit - it oscillates too much near $0$"]
ans = 1
radioq(choices, ans)

###### Question

Find the limit as $x$ goes to $2$ of

$$~
f(x) = \frac{3x^2 - x -10}{x^2 - 4}
~$$

In [ ]:
f(x) = (3x^2 - x - 10)/(x^2 - 4);
val = convert(Float64, N(limit(f, 2)))
numericq(val)

###### Question

Find the limit as $x$ goes to $-2$ of

$$~
f(x) = \frac{\frac{1}{x} + \frac{1}{2}}{x^3 + 8}
~$$

In [ ]:
f(x) = ((1/x) + (1/2))/(x^3 + 8)
numericq(-1/48, .001)

###### Question

Find the limit as $x$ goes to $27$ of

$$~
f(x) = \frac{x - 27}{x^{1/3} - 3}
~$$

In [ ]:
f(x) = (x - 27)/(x^(1//3) - 3)
val = N(limit(f, 27))
numericq(val)

###### Question

Find the limit

$$~
L = \lim_{x \rightarrow \pi/2} \frac{\tan (2x)}{x - \pi/2}
~$$

In [ ]:
f(x) = tan(2x)/(x-PI/2)
val = N(limit(f, PI/2))
numericq(val)

###### Question

The limit of $\sin(x)/x$ at $0$ has a numeric value. This depends upon
the fact that $x$ is measured in radians. Try to find this limit:
`limit(sind(x)/x, x=>0)`. What is the value?

In [ ]:
choices = [q"0", q"1", q"pi/180", q"180/pi"]
ans = 3
radioq(choices, ans)

What is the limit `limit(sinpi(x)/x, x=>0)`?

In [ ]:
choices = [q"0", q"1", q"pi", q"1/pi"]
ans = 3
radioq(choices, ans)

###### Question <small>limit properties</small>

There are several properties of limits that allow one to break down
more complicated problems into smaller subproblems. For example,

$$~
\lim (f(x) + g(x)) = \lim f(x) + \lim g(x)
~$$

is notation to indicate that one can take a limit of the sum of two
function or take the limit of each first, then add and the answer will
be unchanged, provided all the limits in question exist.

Use one or the either to find the limit of $f(x) = \sin(x) + \tan(x) +
\cos(x)$ as $x$ goes to $0$.

In [ ]:
f(x) = sin(x) + tan(x) + cos(x)
numericq(f(0), 1e-5)

###### Question

Does this function have a limit as $h$ goes to $0$ from the right
(that is, assume $h>0$)?

$$~
\frac{h^h - 1}{h}
~$$

In [ ]:
choices = [
"Yes, the value is `-9.2061`",
"Yes, the value is `-11.5123`",
"No, the value heads to negative infinity"
];
ans = 3;
radioq(choices, ans)

###### Question

Compute the limit

$$~
\lim_{x \rightarrow 1} \frac{x}{x-1} - \frac{1}{\log(x)}.
~$$

In [ ]:
f(x) = x/(x-1) - 1/log(x)
val = convert(Float64, N(limit(f, 1)))
numericq(val)

###### Question

Compute the limit

$$~
\lim_{x \rightarrow 1/2} \frac{1}{\pi} \frac{\cos(\pi x)}{1 - (2x)^2}.
~$$

In [ ]:
f(x) =  1/PI * cos(PI*x)/(1 - (2x)^2)
val = N(limit(f, 1//2))
numericq(val)